# Carte avec des centrales nucleaires
### Source : https://www.data.gouv.fr/fr/datasets/centrales-de-production-nucleaire-dedf-sa/

In [1]:
import requests
import json

In [2]:
# Authentication
# https://data.rte-france.com/documents/20182/22648/FR_GuideOauth2_v5.1.pdf/b02d3246-98bc-404c-81c8-dffaad2f1836

BASE_URL = 'http://digital.iservices.rte-france.com/token/oauth/'
password_base64 = "MTNlNmUyZDUtOWE1Ni00NTgyLWEwOGUtZDc4MTcyOWQ1YWE3OjU4OTU5OGQ5LTZjYTEtNGUzZS1iZmVkLThjMDg2ZTgyZTE1OA=="

header = {
    "Authorization": "Basic " + password_base64,
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(f"{BASE_URL}", headers = header)
json_formatted_str = json.dumps(response.json(), indent=2)

print(json_formatted_str)

{
  "access_token": "7q2xA7a91kePq40ZzLI7s7J11yNxqy7naoFk5taNZr9v8WIu64AQWp",
  "token_type": "Bearer",
  "expires_in": 7200
}


In [3]:
access_token = response.json()['access_token']

In [4]:
from datetime import datetime, timedelta, date

def date_to_iso(date):
    return (date.isoformat('T','microseconds')).split('.')[0]+(date.isoformat('T','microseconds')).split('.')[1]

In [5]:
# Scraping example
# Guide : https://data.rte-france.com/catalog/-/api/doc/user-guide/Actual+Generation/1.1

BASE_URL = 'https://digital.iservices.rte-france.com/open_api/actual_generation/v1/actual_generations_per_unit'

now = datetime.now()
start_dt = datetime(now.year, now.month, now.day)
day = timedelta(days = 7)

header = {
    "Authorization": "Bearer " + access_token
}

query_params = {
}


response = requests.get(f"{BASE_URL}", headers = header, params=query_params)

print(response.json())

{'actual_generations_per_unit': [{'start_date': '2023-06-21T00:00:00+02:00', 'end_date': '2023-06-22T00:00:00+02:00', 'unit': {'eic_code': '17W000001023725I', 'name': 'FR_LANDIVISIAUGU', 'production_type': 'FOSSIL_GAS'}, 'values': [{'start_date': '2023-06-21T00:00:00+02:00', 'end_date': '2023-06-21T01:00:00+02:00', 'updated_date': '2023-06-21T01:22:09+02:00', 'value': 413}, {'start_date': '2023-06-21T01:00:00+02:00', 'end_date': '2023-06-21T02:00:00+02:00', 'updated_date': '2023-06-21T02:26:48+02:00', 'value': 414}, {'start_date': '2023-06-21T03:00:00+02:00', 'end_date': '2023-06-21T04:00:00+02:00', 'updated_date': '2023-06-21T04:25:50+02:00', 'value': 420}, {'start_date': '2023-06-21T04:00:00+02:00', 'end_date': '2023-06-21T05:00:00+02:00', 'updated_date': '2023-06-21T05:26:06+02:00', 'value': 419}, {'start_date': '2023-06-21T05:00:00+02:00', 'end_date': '2023-06-21T06:00:00+02:00', 'updated_date': '2023-06-21T06:26:01+02:00', 'value': 420}, {'start_date': '2023-06-21T06:00:00+02:00',

In [6]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 99.9 MB/s eta 0:00:00


In [7]:
# Downloading the geoJSON from the government data

url = 'https://www.data.gouv.fr/fr/datasets/r/2e01aa22-7145-4deb-a2b2-54213774ba0e'
r = requests.get(url, allow_redirects=True)

open('centrales-de-production-nucleaire-edf.geojson', 'wb').write(r.content)

61498

In [8]:
import geopandas as gpd
import pandas as pd

# Adding the position info
locs = gpd.read_file('centrales-de-production-nucleaire-edf.geojson')

numbers = locs.groupby('centrale').cumcount().apply(lambda x: str(x+1))

locs['centrale'] = locs['centrale'].apply(lambda x: x.split(' ')[0])
locs['reacteur'] = locs['centrale'].apply(lambda x: x.split(' ')[0]) + ' ' + numbers

locs = locs[['reacteur','centrale', 'puissance_installee','geometry']].copy()

In [9]:
nuke_prod = {}
for entry in response.json()['actual_generations_per_unit']:
  if entry['unit']['name'] in list(locs['reacteur']):
    nuke_prod[entry['unit']['name']] = entry
    # To delete the unit key
    nuke_prod[entry['unit']['name']].pop('unit', 1)

In [10]:
indexes = []
values = []
reacteurs =[]

for centrale in nuke_prod.keys():
  for obs in nuke_prod[centrale]['values']:
    indexes.append(datetime.strptime(obs['start_date'], "%Y-%m-%dT%H:%M:%S+02:00"))
    values.append(obs['value'])
    reacteurs.append(centrale)

nuke_df = pd.DataFrame(data={"reacteur": reacteurs, 'Date': indexes,'Puissance en MW': values})

nuke_df = nuke_df.merge(locs,how = 'left', on = "reacteur")
nuke_df

,reacteur,Date,Puissance en MW,centrale,puissance_installee,geometry
0,BELLEVILLE 1,2023-06-21 00:00:00,1248.0,BELLEVILLE,1310,POINT (2.87568 47.50895)
1,BELLEVILLE 1,2023-06-21 01:00:00,1248.5,BELLEVILLE,1310,POINT (2.87568 47.50895)
2,BELLEVILLE 1,2023-06-21 02:00:00,1247.5,BELLEVILLE,1310,POINT (2.87568 47.50895)
3,BELLEVILLE 1,2023-06-21 03:00:00,1249.5,BELLEVILLE,1310,POINT (2.87568 47.50895)
4,BELLEVILLE 1,2023-06-21 04:00:00,1250.0,BELLEVILLE,1310,POINT (2.87568 47.50895)
...,...,...,...,...,...,...
670,TRICASTIN 4,2023-06-21 10:00:00,922.0,TRICASTIN,915,POINT (4.73154 44.32635)
671,TRICASTIN 4,2023-06-21 11:00:00,921.5,TRICASTIN,915,POINT (4.73154 44.32635)
672,TRICASTIN 4,2023-06-21 12:00:00,921.0,TRICASTIN,915,POINT (4.73154 44.32635)
673,TRICASTIN 4,2023-06-21 13:00:00,920.5,TRICASTIN,915,POINT (4.73154 44.32635)


In [11]:
centr_groups = []

for centrale in nuke_df['centrale'].unique():
  temp = [centrale]
  reacts = []
  for row in nuke_df.iterrows():
    if (row[1]['centrale'] == centrale) and (row[1]['reacteur'] not in reacts) :
      reacts.append(row[1]['reacteur'])
  if reacts:
    temp.append(reacts)

  centr_groups.append(temp)

dictionnaire_centrales = {}

for centrale in centr_groups:
    nom = centrale[0]
    reacteurs = centrale[1]
    dictionnaire_centrales[nom] = reacteurs

In [12]:
from datetime import datetime
import altair as alt
# Help : https://colab.research.google.com/github/uwdata/visualization-curriculum/blob/master/altair_introduction.ipynb

def to_chart (centrales):
  indexes = []
  values = []
  reacteurs = []

  for centrale in centrales:
    for obs in nuke_prod[centrale]['values']:
      indexes.append(datetime.strptime(obs['start_date'], "%Y-%m-%dT%H:%M:%S+02:00"))
      values.append(obs['value'])
      reacteurs.append(centrale)

  df = pd.DataFrame(data={"Reacteur": reacteurs, 'Date': indexes,'Puissance en MW': values})

  chart = alt.Chart(df,
                  mark = 'line').encode(x='Date',
                                        y=alt.Y('Puissance en MW',
                                                scale=alt.Scale(domain=[df['Puissance en MW'].min()-20,
                                                                        df['Puissance en MW'].max()+20])),
                                        color='Reacteur')#.properties(title= centrale)
  #chart.display()
  return chart.to_json()

for centrale in dictionnaire_centrales.keys():
  to_chart(dictionnaire_centrales[centrale])

In [13]:
import folium
from folium import Vega, features

title_html = '''
             <h3 style="font-size:16px"><b>{}</b></h3>
             '''.format("Centrales Nucleaires en France")

fig = folium.Map(width=1250, height=800)

fig.get_root().html.add_child(folium.Element(title_html))

for centrale in dictionnaire_centrales.keys():
    entries = nuke_df[nuke_df['centrale'] == centrale].reset_index(drop=True)
    vis1 = to_chart(dictionnaire_centrales[centrale])
    folium.Marker(
    (entries['geometry'][0].y, entries['geometry'][0].x),
    popup=folium.Popup(max_width=750).add_child(folium.VegaLite(vis1)),
    tooltip=f"{entries['centrale'][0]}"
    ).add_to(fig)

fig.fit_bounds(fig.get_bounds(), padding=(30, 30))

fig

# Pour la suite
L'idée c'est de calculer l'énergie totale produite.

https://stackoverflow.com/questions/36583308/using-scipy-to-calculate-integral-over-time-series-data-without-function